In [1]:
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset, num2date

In [6]:
path = "../data/ccmp/2010/01/analysis_20100103_v11l30flk.nc"

In [7]:
nc = Dataset(path, 'r')

In [8]:
print nc.variables.keys()

[u'lon', u'lat', u'time', u'uwnd', u'vwnd', u'nobs']


In [12]:
print nc.variables['time'].ncattrs()

[u'units', u'long_name', u'actual_range', u'delta_t', u'avg_period']


In [13]:
print nc.variables['time'].units

hours since 1987-01-01 00:00:0.0


In [14]:
time = num2date(nc.variables['time'][:], nc.variables['time'].units)

In [15]:
lat = nc.variables['lat'][:]
lon = nc.variables['lon'][:]
u = nc.variables['uwnd'][:]
v = nc.variables['uwnd'][:]

In [16]:
w = np.sqrt(u**2 + v**2)

In [21]:
print time.shape
print lat.shape
print lon.shape
print w.shape

(4,)
(628,)
(1440,)
(4, 628, 1440)
